# Testing C++ code in python
June 12, 2020 \
June 20, 2020 : Added test for batch of images with multi-channels

In [1]:
import numpy as np

In [2]:
xsize,ysize=128,128
# xsize,ysize=10,10
batch_size,num_channels=100,2

## Save image for C++ code

In [ ]:
# ### Save single 2D image to csv for C++ code to handle
fname='data/gen_images.npy'
a1=np.load(fname)
print(a1.shape)

a2=a1[0:batch_size*num_channels,0,:xsize,:ysize]
print(a2.shape)
np.savetxt('data/images.csv',a2.flatten(),delimiter=',',newline=',')

### Save second image
a2=a1[1*batch_size*num_channels:2*batch_size*num_channels,0,:xsize,:ysize]
print(a2.shape)
np.savetxt('data/images2.csv',a2.flatten(),delimiter=',',newline=',')

### Test for single image

In [ ]:
fname='data/images.csv'
x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(xsize,ysize)
print(x.shape)
# x

In [ ]:
y1=np.fft.fft2(x)
y2=abs(y1)
# y2

# for i in y2: 
#     for j in i:
#         print(np.around(j,decimals=6),end='\t')
#     print('')

z1=f_radial_profile(y2)
print(z1)

In [ ]:
### Read c++ file
cp_ip_file='data/op.csv'
# x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(5,5)
z_cpp=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64)
print(z_cpp)

In [ ]:
### Check arrays
np.allclose(z1,z_cpp[:z1.shape[0]],rtol=1e-4,atol=1e-8)

In [ ]:
### Check manually elementwise
for i in range(z1.shape[0]):
    if (np.around(z1[i],4)!=np.around(z_cpp[i],4)): 
          print(i,z1[i],z_cpp[i])

## Multi-channel batch example

### Read input

In [ ]:
fname='data/images.csv'
x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(batch_size,num_channels,xsize,ysize)
print(x.shape)
# x

### Compute batch spectrum

In [6]:
def f_radial_profile(data, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    if center[0]==None and center[1]==None:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0]) # compute centers
        
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    
    return radialprofile

def f_compute_spectrum(arr):
    y1=np.fft.fft2(arr)
    y2=abs(y1)
    z1=f_radial_profile(y2)
    return(z1)
    
def f_compute_batch_spectrum(arr):
    batch_pk=np.array([f_compute_spectrum(i) for i in arr])
    return batch_pk


### Code ###
def f_image_spectrum(x):
    '''
    Data has to be in the form (batch,channel,x,y)
    '''
    print(x.shape)
    mean=[[] for i in range(num_channels)]    
    sdev=[[] for i in range(num_channels)]    

    for i in range(num_channels):
        arr=x[:,i,:,:]
#         print(i,arr.shape)
        batch_pk=f_compute_batch_spectrum(arr)
#         print(batch_pk)
        mean[i]=np.mean(batch_pk,axis=0)
        sdev[i]=np.std(batch_pk,axis=0)
    mean=np.array(mean)
    sdev=np.array(sdev)
    return mean,sdev


In [ ]:
mean,sdev=f_image_spectrum(x)
print(mean.shape,sdev.shape)

In [ ]:
# print(mean,sdev)

### Read from c++ files and compare

In [ ]:
### Read c++ files
cp_ip_file='data/op_spec_mean.csv'
# x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(5,5)
z_cpp1=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(num_channels,-1)

cp_ip_file='data/op_spec_sdev.csv'
# x=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(5,5)
z_cpp2=np.loadtxt(cp_ip_file,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(num_channels,-1)
# print(z_cpp1,'\n',z_cpp2)

In [ ]:
### Check arrays
print(np.allclose(mean,z_cpp1[:,:mean.shape[1]],rtol=1e-4,atol=1e-8))

print(np.allclose(sdev,z_cpp2[:,:sdev.shape[1]],rtol=1e-4,atol=1e-8))

## Conclusion:
###### July 2, 2020
Reading input + computing FFT + computing Modulus + Getting Radial profile \
Arrays from python and C++ match !!\
The C++ code is correct and ready to port to LBANN

###### July 20, 2020
Reading input + computing FFT + computing Modulus + Getting Radial profile + computing mean, sdev \
Arrays from python and C++ match !!\


## Testing spectral loss

In [4]:
fname='data/images.csv'
x1=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(batch_size,num_channels,xsize,ysize)
fname='data/images2.csv'
x2=np.loadtxt(fname,delimiter=',',dtype=str)[:-1].astype(np.float64).reshape(batch_size,num_channels,xsize,ysize)



In [7]:
mean1,sdev1=f_image_spectrum(x1)
mean2,sdev2=f_image_spectrum(x2)


(100, 2, 128, 128)
(100, 2, 128, 128)


In [8]:
mean1.shape

(2, 90)

In [9]:
k_crop=int(xsize/2) ## =64
np.log(np.mean(np.square(mean1[:,:k_crop]-mean2[:,:k_crop]))),np.log(np.mean(np.square(sdev1[:,:k_crop]-sdev2[:,:k_crop])))

(-4.318593843821501, -5.298694316092033)

### Conclusion:
July 28,2020

Results matches c++ code output exactly
